In [2]:
import polars as pl
import glob 
import seaborn as sns
import plotly.express as px
import pyarrow as pyarrow

decoded_files = glob.glob("pos_ct/*/*.decoded.hap*.txt")

In [3]:
# create list for storing df from each file 
dfs = []

# for each file extract pop and ID from the file name and add to each df 
for file in decoded_files:
    file_name = file.split('.')[0]
    ind_id = file_name.split('/')[2]
    pop = file_name.split('/')[1]
    
    df = pl.read_csv(file, has_header=True, separator='\t')
    
    # Adding ID and pop to each df
    df_1 = df.with_columns( # with _columns to add columns to a data frame 
        (pl.lit(ind_id).alias("ID")), # pl.lit returns literal value pl.alias to name a column
        (pl.lit(pop).alias("pop")),
        (pl.col("end") + 1000) # adding 1000 to the end coordinate to match fragment length 
        )
    # print(df_1.head(5))

    dfs.append(df_1) # adding df to the dfs list 

# concatenating dfs from the list using align option to align dfs by column names 
decoded_df = pl.concat(dfs, how='align')
decoded_df.head(5)

chrom,start,end,length,state,mean_prob,snps,ID,pop
i64,i64,i64,i64,str,f64,i64,str,str
1,0,1000,1000,"""Archaic""",0.51065,0,"""HG00698""","""CHS"""
1,0,2000,2000,"""Archaic""",0.51407,0,"""HG00674""","""CHS"""
1,0,7000,7000,"""Archaic""",0.54458,0,"""HG00442""","""CHS"""
1,0,11000,11000,"""Archaic""",0.56568,0,"""HG00566""","""CHS"""
1,0,15000,15000,"""Archaic""",0.61127,0,"""HG00707""","""CHS"""


In [4]:
def interval(prob: float) -> str:
    """Define a function for binning mean_prob into intervals 
    """
    if prob >= 0.5 and prob < 0.6:
        return("[0.5-0.6)")
    elif prob >= 0.6 and prob < 0.7:
        return("[0.6-0.7)")
    elif prob >= 0.7 and prob < 0.8:
        return("[0.7-0.8)")
    elif prob >= 0.8 and prob < 0.9:
        return("[0.8-0.9)")
    elif prob >= 0.9 and prob < 0.95:
        return("[0.9-0.95)")
    elif prob >= 0.95 and prob < 0.99:
        return("[0.95-0.99)")
    elif prob >= 0.99 and prob < 1:
        return("[0.99-1)")
    elif prob == 1:
        return("1")


In [ ]:
# Archaic only df with added column of posterior probability intervals for further processing. Without snp count and coordinates. 
archaic_df = (decoded_df.filter(pl.col("state") == "Archaic")
              .select(["pop", "ID", "length", "mean_prob"])
              .with_columns(
                  (pl.col("mean_prob").apply(lambda x: interval(x)).alias("interval_prob"),
                   pl.col("mean_prob").round(1).alias("rounded_mean_prob"))
))
# print(archaic_df.head(20))

# Plotting a histogram of fragment length against rounded mean_prob
# px.histogram(archaic_df, x="length", color="rounded_mean_prob", log_x=True, 
#             labels={"rounded_mean_prob":"rounded posterior probability"}, 
#             title="Fragment length distribution (log-scaled) based on posterior probability cut-off",
#             opacity=0.8)

# Plotting a histogram of fragment count against mean_prob
# px.histogram(archaic_df, x="mean_prob", nbins=50, color="pop", marginal="box", hover_data=archaic_df.columns, 
#              title="Archaic fragment distribution across posterior probability cut-off values", 
#              labels={"mean_prob":"posterior probability", "pop":"populations"}, range_x=(0.5, 1))
 
# Calculating fragment count and fragment length stats in defined posterior probability cutoff interavals
# interval_archaic = archaic_df.groupby(["interval_prob", "pop"]).agg(
#     (pl.count("interval_prob").alias("fragment_count")), 
#     (pl.mean("length").alias("mean_length")), 
#     (pl.median("length").alias("median_length")), 
#     (pl.min("length").alias("min_length")), 
#     (pl.max("length").alias("max_lenght"))
#     ).sort("interval_prob")
# print(interval_archaic)

# Saving interval_archaic dataframe to excel worksheet to produce a table figure 
# interval_archaic.write_excel("pos_ct/excel_output.xlsx", "probability_interval_stat")

In [62]:
# Filter Archaic fragments with posterior probability above (and including) 0.9 
filtered_df = (decoded_df.filter(pl.col("state") == "Archaic")
              .filter(pl.col("mean_prob") >= 0.9)
              .select(["pop", "ID", "chrom", "start", "end", "length"])
              .with_columns(
                  pl.col()
              )
)
print(filtered_df.head(20))


shape: (20, 6)
┌─────┬─────────┬───────┬─────────┬─────────┬────────┐
│ pop ┆ ID      ┆ chrom ┆ start   ┆ end     ┆ length │
│ --- ┆ ---     ┆ ---   ┆ ---     ┆ ---     ┆ ---    │
│ str ┆ str     ┆ i64   ┆ i64     ┆ i64     ┆ i64    │
╞═════╪═════════╪═══════╪═════════╪═════════╪════════╡
│ GBR ┆ HG00234 ┆ 1     ┆ 1104000 ┆ 1128000 ┆ 24000  │
│ GBR ┆ HG00115 ┆ 1     ┆ 1107000 ┆ 1129000 ┆ 22000  │
│ GBR ┆ HG00109 ┆ 1     ┆ 1495000 ┆ 1521000 ┆ 26000  │
│ GBR ┆ HG00128 ┆ 1     ┆ 1495000 ┆ 1521000 ┆ 26000  │
│ …   ┆ …       ┆ …     ┆ …       ┆ …       ┆ …      │
│ CHS ┆ HG00654 ┆ 1     ┆ 1520000 ┆ 1559000 ┆ 39000  │
│ CHS ┆ HG00705 ┆ 1     ┆ 1521000 ┆ 1542000 ┆ 21000  │
│ CHS ┆ HG00598 ┆ 1     ┆ 1521000 ┆ 1559000 ┆ 38000  │
│ GBR ┆ HG00254 ┆ 1     ┆ 1521000 ┆ 1559000 ┆ 38000  │
└─────┴─────────┴───────┴─────────┴─────────┴────────┘


In [ ]:

# Group by pop and ID to calculate average fragment length for each individual
ind_mean_length = filtered_df.groupby(["pop", "ID"]).agg( 
                  avg_length=pl.mean("length"), 
                  fragment_count=pl.count("ID"))
print(ind_mean_length.head(20))

# Group by pop and calculate mean fragment count and length per haploid genome for each population 
pop_average = ind_mean_length.groupby("pop").agg(
    avg_fragment_length=pl.col("avg_length").mean(),
    avg_fragment_count=pl.col("fragment_count").mean(),
    median_fragment_count=pl.col("fragment_count").median(),
    min_fragment_count=pl.col("fragment_count").min(),
    max_fragment_count=pl.col("fragment_count").max(),
    sd_fragment_count=pl.col("fragment_count").std()
)
# Export to excel for table figure creation
# pop_average.write_excel("pos_ct/avg_lenght_and_count.xlsx", "avg_length_and_count_per_genome")

In [60]:
# Calculate unique sequence: using entire population find overlapping fragments and separate them into continuous fragments with cumulative frequency 

# filtered_df.head(20)

unique_count=filtered_df.

unique_count.head(20)

pop,ID,chrom,start,end,length,unique_fragment_count
str,str,i64,i64,i64,i64,u32
"""GBR""","""HG00234""",1,1104000,1128000,24000,259563
"""GBR""","""HG00115""",1,1107000,1129000,22000,259563
"""GBR""","""HG00109""",1,1495000,1521000,26000,259563
"""GBR""","""HG00128""",1,1495000,1521000,26000,259563
"""GBR""","""HG00254""",1,1521000,1559000,38000,259563
"""GBR""","""HG00236""",1,1521000,1559000,38000,259563
"""GBR""","""HG00137""",1,1521000,1559000,38000,259563
"""GBR""","""HG00259""",1,1521000,1559000,38000,259563
"""GBR""","""HG00132""",1,1949000,1965000,16000,259563
